In [1]:
  ! pip install pydlm


In [2]:
from pydlm import dlm, trend, seasonality
import pandas as pd
import numpy as np
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as mat
import random 

sns.set()


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
datam=pd.read_csv("/content/drive/My Drive/hack/Copy of TrainingData - Training.csv")
(datam.head())

,Period,EQ,Social_Search_Impressions,Social_Search_Working_cost,Digital_Impressions,Digital_Working_cost,Print_Impressions.Ads40,Print_Working_Cost.Ads50,OOH_Impressions,OOH_Working_Cost,SOS_pct,Digital_Impressions_pct,CCFOT,Median_Temp,Median_Rainfall,Fuel_Price,Inflation,Trade_Invest,Brand_Equity,Avg_EQ_Price,Any_Promo_pct_ACV,Any_Feat_pct_ACV,Any_Disp_pct_ACV,EQ_Base_Price,Est_ACV_Selling,pct_ACV,Avg_no_of_Items,pct_PromoMarketDollars_Category,RPI_Category,Magazine_Impressions_pct,TV_GRP,Competitor1_RPI,Competitor2_RPI,Competitor3_RPI,Competitor4_RPI,EQ_Category,EQ_Subcategory,pct_PromoMarketDollars_Subcategory,RPI_Subcategory
0,2016 - Period:1,504.784933,NaN,NaN,37148.2,2045.282,NaN,NaN,NaN,NaN,7.446883,NaN,100.0,32.950,0.5150,8.226,0.013258,42744.25684,42.410000,49.091447,9.691794,NaN,1.688896,1.427532,8696587915,39.441025,2.611782,0.0339,35.817030,NaN,NaN,97.173365,35.557371,44.502717,63.377268,1728388.673,331927.5394,0.162732,40.560563
1,2016 - Period:2,490.226477,NaN,NaN,50886.8,2370.584,0.000014,0.000015,NaN,NaN,11.677082,NaN,87.5,34.625,0.2700,7.473,0.009938,36290.43956,42.410000,49.057612,9.415938,NaN,1.687952,1.442716,8682307085,38.417224,2.522814,0.0391,36.389065,NaN,NaN,97.850760,37.223072,45.720077,62.731742,1900859.879,334611.3806,0.231650,40.046301
2,2016 - Period:3,479.244686,NaN,NaN,253333.2,3180.888,0.000016,0.000017,NaN,NaN,0.102858,NaN,96.0,46.700,0.3900,8.001,0.007832,34852.14166,42.410000,49.367514,5.176684,NaN,NaN,1.423408,8706897549,36.499442,2.477322,0.0228,36.599984,NaN,NaN,96.397739,40.800563,41.822953,62.090417,2036436.906,387148.3582,0.125394,40.411115
3,2016 - Period:4,489.057428,NaN,NaN,3426239.0,13595.398,0.000016,0.000019,NaN,NaN,0.249055,NaN,100.0,49.650,0.3500,8.767,0.010034,24869.55899,43.543571,50.657634,6.088273,NaN,NaN,1.443401,8660288592,38.135159,2.520409,0.0147,38.201852,NaN,NaN,98.936519,36.576140,41.578429,63.374358,2113635.013,482489.6740,0.056603,42.213246
4,2016 - Period:5,477.031994,NaN,NaN,552197.8,3983.134,0.000016,0.000019,NaN,NaN,13.338804,NaN,100.0,58.100,0.5025,9.277,0.009546,61675.32162,43.790000,50.625777,7.827112,NaN,0.170731,1.473687,8644518558,38.629564,2.497285,0.0219,38.648805,31.702433,25.3,100.509969,36.032016,42.602412,63.291934,2402211.102,629826.6484,0.065059,43.345578


In [4]:
datam.isnull().any()


Period                                False
EQ                                    False
Social_Search_Impressions              True
Social_Search_Working_cost             True
Digital_Impressions                    True
Digital_Working_cost                   True
Print_Impressions.Ads40                True
Print_Working_Cost.Ads50               True
OOH_Impressions                        True
OOH_Working_Cost                       True
SOS_pct                               False
Digital_Impressions_pct                True
CCFOT                                 False
Median_Temp                           False
Median_Rainfall                       False
Fuel_Price                            False
Inflation                             False
Trade_Invest                          False
Brand_Equity                          False
Avg_EQ_Price                          False
Any_Promo_pct_ACV                     False
Any_Feat_pct_ACV                       True
Any_Disp_pct_ACV                

In [5]:
def removenull(dataq):
  for col in dataq:
    if(col!="Period"):
      num_of_rows=dataq['Period'].count()
      for i in range(0,num_of_rows-2):
        if(pd.isna(dataq.iloc[i][col])):
          f_found=i
          first=i-1
          countna=0
          for j in range(f_found,num_of_rows):
            countna=countna+1
            if(not pd.isna(dataq.iloc[j][col])):
              l_found=j
              print(dataq[col][f_found],dataq[col][l_found])
              dataq[col][f_found:l_found].fillna((dataq.iloc[first][col]+dataq.iloc[l_found][col])/2,inplace=True)
              break
  return dataq
datam=removenull(datam)
datam.fillna(method='bfill',inplace=True)
datam.fillna(method='ffill',inplace=True)

nan 3919100.6
nan 3919100.6
nan 3919100.6
nan 3919100.6
nan 3919100.6
nan 3919100.6
nan 3919100.6
nan 3919100.6
nan 3919100.6
nan 24839.812
nan 24839.812
nan 24839.812
nan 24839.812
nan 24839.812
nan 24839.812
nan 24839.812
nan 24839.812
nan 24839.812
nan 2262738.0
nan 3006588.2
nan 31259.4159
nan 36791.8245
nan 1.4000000000000001e-05
nan 1.4999999999999999e-05
nan 63009885.0
nan 63009885.0
nan 63009885.0
nan 63009885.0
nan 63009885.0
nan 63009885.0
nan 63009885.0
nan 63009885.0
nan 63009885.0
nan 63009885.0
nan 63009885.0
nan 63009885.0
nan 63009885.0
nan 63009885.0
nan 63009885.0
nan 63009885.0
nan 1876621874.0
nan 300082.9333
nan 300082.9333
nan 300082.9333
nan 300082.9333
nan 300082.9333
nan 300082.9333
nan 300082.9333
nan 300082.9333
nan 300082.9333
nan 300082.9333
nan 300082.9333
nan 300082.9333
nan 300082.9333
nan 300082.9333
nan 300082.9333
nan 300082.9333
nan 5224427.6
nan 3.828396226
nan 3.828396226
nan 3.828396226
nan 3.828396226
nan 3.828396226
nan 3.828396226
nan 3.8283962

In [6]:
dropping_columns={"pct_PromoMarketDollars_Category",#       0.292553
"Brand_Equity",#                          0.275284
"CCFOT",#                                  0.247963
"Any_Disp_pct_ACV",#                       0.246106
"Median_Temp",#                            0.165922
"pct_PromoMarketDollars_Subcategory",#     0.143906
"RPI_Subcategory",#                        0.055342
"TV_GRP",#                                -0.040370
"Print_Working_Cost.Ads50",#              -0.049174
"RPI_Category",#                          -0.152428
"Print_Impressions.Ads40",#               -0.177558
"Social_Search_Working_cost",#            -0.188550
"Competitor4_RPI",#                       -0.195253
"SOS_pct",#                               -0.214566
"Social_Search_Impressions",#             -0.239020
"Magazine_Impressions_pct",#              -0.266168
"Competitor1_RPI"}
datam.drop(columns=dropping_columns)

,Period,EQ,Digital_Impressions,Digital_Working_cost,OOH_Impressions,OOH_Working_Cost,Digital_Impressions_pct,Median_Rainfall,Fuel_Price,Inflation,Trade_Invest,Avg_EQ_Price,Any_Promo_pct_ACV,Any_Feat_pct_ACV,EQ_Base_Price,Est_ACV_Selling,pct_ACV,Avg_no_of_Items,Competitor2_RPI,Competitor3_RPI,EQ_Category,EQ_Subcategory
0,2016 - Period:1,504.784933,37148.2,2045.282000,6.300988e+07,3.000829e+05,3.828396,0.5150,8.226,0.013258,42744.25684,49.091447,9.691794,1.633989,1.427532,8696587915,39.441025,2.611782,35.557371,44.502717,1728388.673,331927.5394
1,2016 - Period:2,490.226477,50886.8,2370.584000,6.300988e+07,3.000829e+05,3.828396,0.2700,7.473,0.009938,36290.43956,49.057612,9.415938,1.633989,1.442716,8682307085,38.417224,2.522814,37.223072,45.720077,1900859.879,334611.3806
2,2016 - Period:3,479.244686,253333.2,3180.888000,6.300988e+07,3.000829e+05,3.828396,0.3900,8.001,0.007832,34852.14166,49.367514,5.176684,1.633989,1.423408,8706897549,36.499442,2.477322,40.800563,41.822953,2036436.906,387148.3582
3,2016 - Period:4,489.057428,3426239.0,13595.398000,6.300988e+07,3.000829e+05,3.828396,0.3500,8.767,0.010034,24869.55899,50.657634,6.088273,1.633989,1.443401,8660288592,38.135159,2.520409,36.576140,41.578429,2113635.013,482489.6740
4,2016 - Period:5,477.031994,552197.8,3983.134000,6.300988e+07,3.000829e+05,3.828396,0.5025,9.277,0.009546,61675.32162,50.625777,7.827112,1.633989,1.473687,8644518558,38.629564,2.497285,36.032016,42.602412,2402211.102,629826.6484
5,2016 - Period:6,487.855299,29892.2,1800.344000,6.300988e+07,3.000829e+05,3.828396,0.4600,9.824,0.009290,40458.59237,51.268984,9.791997,1.633989,1.457055,8627353001,39.097729,2.455277,36.996437,44.943022,2796949.720,806075.7642
6,2016 - Period:7,466.399281,6021768.0,7591.644000,6.300988e+07,3.000829e+05,3.828396,0.4250,9.642,0.008284,40945.74781,50.676333,8.802657,3.385373,1.437953,8662698543,39.017768,2.412757,37.314308,43.358409,2867306.933,828776.1488
7,2016 - Period:8,546.053084,1815307.2,7866.572000,6.300988e+07,3.000829e+05,3.828396,0.5550,9.148,0.008461,22026.52625,48.992972,13.660056,5.136756,1.453832,8511934271,37.940027,2.641764,34.013304,41.356740,2813494.081,789109.5698
8,2016 - Period:9,464.925607,153633.2,3579.130000,6.300988e+07,3.000829e+05,3.828396,0.5600,9.225,0.011022,53214.88476,49.308852,9.791260,5.154291,1.476185,7928216512,35.696189,2.559809,37.377758,40.577016,2441916.655,606656.5082
9,2016 - Period:10,357.648693,30331.0,23205.988710,6.300988e+07,3.000829e+05,3.828396,0.2950,9.331,0.013966,25343.21025,54.138214,6.060160,5.356933,1.525389,7765786227,32.618780,2.664421,46.598751,44.631443,2017353.844,421134.3688


In [0]:
df_differenced=datam

df_differenced.drop(columns={'Period','OOH_Impressions','Any_Feat_pct_ACV','OOH_Working_Cost','Digital_Impressions_pct','Trade_Invest'},inplace=True)

In [0]:
linear_trend = trend(degree=1, discount=0.95, name='linear_trend', w=10)
# A seasonality
seasonal52 = seasonality(period=52, discount=0.99, name='seasonal52', w=10)
# Build a simple dlm
simple_dlm = dlm(df_differenced) + linear_trend + seasonal52

In [9]:
simple_dlm.fit()
# Plot the fitted results
simple_dlm.turnOff('data points')
simple_dlm.plot()

Initializing models...
Initialization finished.
Starting forward filtering...


UFuncTypeError: ignored